<a href="https://colab.research.google.com/github/Jeidsgn/Frame_overlay/blob/main/Frame_overlay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conectarse con Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
directorio = '/content/drive/MyDrive/backup/Docu/Diseño/anime/Experimen2/filtro'
resultado = directorio + '/multiplicación'
carpetas = os.listdir(directorio)
#Crear carpeta resultados si no existe
if not os.path.exists(resultado):
  os.makedirs(resultado)
else:
  # Obtenemos la lista de archivos de la carpeta
  archivos = os.listdir(resultado)
  # Recorremos cada archivo y lo eliminamos
  for archivo in archivos:
    os.remove(resultado + '/' + archivo)

FileNotFoundError: ignored

In [ ]:
# Hacer transparente un color específico
def trans(img):
    # Comprobar el número de canales de la imagen
    if img.shape[2] == 4:
        # Si tiene 4 canales, separar los canales RGB y alpha
        b,g,r,alpha = cv2.split(img)
    elif img.shape[2] == 3:
        # Si tiene 3 canales, separar los canales RGB y calcular el canal alpha
        b,g,r = cv2.split(img)
        alpha = (255 - np.mean([b,g,r], axis=0)).astype(np.uint8)
    else:
        raise ValueError("La imagen debe tener 3 o 4 canales")
    # Unir la imagen y el canal alfa
    img_alpha = cv2.merge((b,g,r,alpha))
    return img_alpha

def imprimir_porcentaje(iteracion, total_fotogramas):
    porcentaje = (iteracion / total_fotogramas) * 100
    print("\rProgreso: {:.2f}%".format(porcentaje), end="")

frames = []
for carpeta in carpetas:
    if carpeta.startswith("out_"):
        fotogramas_carpeta = os.listdir(os.path.join(directorio,carpeta))
        frames.extend(fotogramas_carpeta)
f = len(frames)

In [ ]:
# Lista de colores RGB deseados
colors = [[255, 255, 255], #blanco
          [0, 0, 0], #negro
          [255, 55, 0], #azul rey
          [255, 120, 0], #azul claro
          [0, 0, 160], #vinotinto
          [0, 130, 0], #Verde cali
          [0, 130, 255], #Amarillo
          [200, 255, 255], #Beige
          [0, 160, 0], #Verde claro
          [255, 255, 0], #Azul celeste
          [0, 0, 255], #Rojo
          [0, 90, 255], #Naranja
          [255, 120, 80], #Morado claro
          [0, 100, 190], #Amarillo quemado
          [255, 40, 100], #Morado
          [150, 0, 255], #Fucsia
          [200, 160, 255], #Rosa
          [70, 0, 0], #Café
          [60, 0, 0], #Azul oscuro
          ]

In [ ]:
directorio = '/content/drive/MyDrive/backup/Docu/Diseño/anime/Experimen2/filtro'
resultado = directorio + '/multiplicación'
carpetas = os.listdir(directorio)
i=0
carpetas.sort()
for carpeta in carpetas:
  # Verificamos si la carpeta cumple con el patrón "out_[#####]"
  if 'out_' in carpeta:
    # Obtenemos el número del fotograma key
    numero = carpeta.split('_')[1]
    # Abrimos los fotogramas de la carpeta
    fotogramas = os.listdir(directorio + '/' + carpeta)
    # Abrimos el primer fotograma para obtener sus dimensiones
    img = cv2.imread(directorio + '/' + carpeta + '/' + fotogramas[0])
    # Obtenemos el menor y mayor fotograma, primero la lista de nombres
    numeros = []
    # Pasamos nombres a números
    for fotograma in fotogramas:
      nombre = int(fotograma.split('.')[0])
      numeros.append(nombre)
    # Obtenemos el menor y mayor fotograma
    numero_menor = min(numeros)
    numero_mayor = max(numeros)
    # Recorremos cada fotograma
    for fotograma in fotogramas:
         
      # Abrimos el fotograma
      img = cv2.imread(directorio + '/' + carpeta + '/' + fotograma)
      # revisamos si existe ya ese fotograma en resultados
      if os.path.exists(resultado + '/' + fotograma):
        # Si existe, usamos ese como base
        img_base = cv2.imread(resultado + '/' + fotograma)
      else:
        # si no, usamos el fotograma abierto
        img_base = img
      if int(numero)<int(fotograma.split('.')[0]):
        # Calculamos la distancia máxima entre el fotograma key y el fotograma mayor
        distancia_máxima = abs(numero_mayor - int(numero))
        # Calculamos la distancia del fotograma actual al fotograma key
        distancia_al_key = abs(int(numero) - int(fotograma.split('.')[0]))
        # Calculamos la opacidad de manera lineal
        opacidad = 1-(distancia_al_key / distancia_máxima)
      elif int(numero)>int(fotograma.split('.')[0]):
        # Calculamos la distancia máxima entre el fotograma key y el fotograma menor
        distancia_máxima = abs(numero_menor - int(numero))
        # Calculamos la distancia del fotograma actual al fotograma key
        distancia_al_key = abs(int(numero) - int(fotograma.split('.')[0]))
        # Calculamos la opacidad de manera lineal
        opacidad = 1-(distancia_al_key / (distancia_máxima))  
      else:
        opacidad = 1
      # Aplicamos la opacidad a la imagen
      img_resultado = cv2.addWeighted(img_base, 1-opacidad, img, opacidad, 0)
      
      #cv2_imshow(img_resultado)
      # Guardamos la imagen resultante en la carpeta resultados
      cv2.imwrite(resultado + '/' + fotograma, img_resultado)
      # Porcentaje
      i += 1
      imprimir_porcentaje(i, f)

print('Proceso terminado')